In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import awkward as ak

In [2]:
filename = 'pionplus_1k.npy'
file = open(filename, 'rb')
mc_truth = np.load(file,allow_pickle=True) #Added Generator E and Angle
data = np.load(file,allow_pickle=True)

test_file = open('test.npy', 'rb')
test_data = np.load(test_file,allow_pickle=True)

In [3]:
print("----------------- ORIGINAL test.npy -------------------")
print('number of images in event', len(test_data[0]))

----------------- ORIGINAL test.npy -------------------
number of images in event 400


In [4]:
print("--------------- NEW .npy ----------------")
print('number of images in event', len(data[0]))
#print(data)

--------------- NEW .npy ----------------
number of images in event 400


In [5]:
mc_truth.item().get('true_energy')

20.007061

In [6]:
#============ Original ============================

X = []
Y = []
with open('test.npy', 'rb') as f:#wrong file for now
#with open(filename, 'rb') as f:
    data = np.load(f,allow_pickle=True)
    ievt = 0
    print('number of images in event', len(data[ievt]))
    ## Here we loop over all "images", which are created by integrating HCAL sampling layers defing 3 sectors defined 
    ## by two z position that define boundary. Note for all images ECAL is the same (no longitudinal separation in ECAL)
    for im in range(len(data[ievt])):
        event = np.c_[data[ievt][im]['HCAL1_x'][0],data[ievt][im]['HCAL1_y'][0],data[ievt][im]['HCAL1_E'][0]]
        depths = np.array([2*np.ones(len(data[ievt][im]['HCAL1_x'][0]))]).flatten()
        if (len(data[ievt][im]['HCAL2_x'])>0):
            event = np.concatenate([event,np.c_[data[ievt][im]['HCAL2_x'][0],data[ievt][im]['HCAL2_y'][0],data[ievt][im]['HCAL2_E'][0]]])
            depths = np.concatenate([depths,np.array([3*np.ones(len(data[ievt][im]['HCAL2_x'][0]))]).flatten()])
        if (len(data[ievt][im]['HCAL3_x'])>0):
            event = np.concatenate([event,np.c_[data[ievt][im]['HCAL3_x'][0],data[ievt][im]['HCAL3_y'][0],data[ievt][im]['HCAL3_E'][0]]])
            depths = np.concatenate([depths,np.array([4*np.ones(len(data[ievt][im]['HCAL3_x'][0]))]).flatten()])
        event = np.concatenate([event,np.c_[data[ievt][im]['ECAL_x'],data[ievt][im]['ECAL_y'],data[ievt][im]['ECAL_E']]])
        depths = np.concatenate([depths,np.array([1*np.ones(len(data[ievt][im]['ECAL_x']))]).flatten()])
        event = np.insert(event, 3, depths,axis=1)
        event = np.insert(event, 4, data[ievt][im]['boundary'][0]*np.ones(len(event)),axis=1) #better to make these global features, but I did not want to download the latest energyflow package
        event = np.insert(event, 5, data[ievt][im]['boundary'][1]*np.ones(len(event)),axis=1)
        X += [event]
        trueenergy = 1. #Miguel, please add this!
        Y += [trueenergy]
X = np.array(X)
Y = np.array(Y)
print(np.shape(X))

number of images in event 400
(400, 96, 6)


In [7]:
def get_max_ncell():
    print("This function should get the largest size cells, and make a large NDarry, filled with zeros")
    #this route means that outside of the image loop, the event array should be inserted into all_events array

In [72]:
X = []
Y = []
X = ak.ArrayBuilder()
X.begin_list()
#builder.begin_list()
#builder.append(x)
#builder.append(y)
#builder.append(z)
#builder.end_list()
with open(filename, 'rb') as f:
#with open("test.npy", 'rb') as f:
    mc_truth = np.load(f,allow_pickle=True)
    data = np.load(f,allow_pickle=True)
    
    print('number of events: ', len(data))
    print('number of "images" per event: ', len(data[0]))
    
    ievt=0
    #if (ievt!=-99): #place holder to maintain indents...
    #for ievt in range(0,len(data)):
    for ievt in range(0,2):
        
        ## Here we loop over all events and then all "images", which are created by integrating HCAL sampling layers defing 3 sectors defined 
        ## by two z position that define boundary. Note for all images ECAL is the same (no longitudinal separation in ECAL)

        for im in range(len(data[ievt])):
            #FIXME: assumes ECAL is always hit
            event = np.c_[data[ievt][im]['ECAL_x'],data[ievt][im]['ECAL_y'],data[ievt][im]['ECAL_E']]
            depths = np.array([1*np.ones(len(data[ievt][im]['ECAL_x']))]).flatten()
            
            if (ievt == im == 0):
                print("ECAL X:", np.shape(data[ievt][im]['ECAL_x']), "+ ECAL Y", np.shape(data[ievt][im]['ECAL_y']),
                  "+ ECAL E", np.shape(data[ievt][im]['ECAL_E']),"=",np.shape(event), "[using np.c_]")

            if (len(data[ievt][im]['HCAL1_x'])>0):
                event = np.concatenate([event,np.c_[data[ievt][im]['HCAL1_x'][0],data[ievt][im]['HCAL1_y'][0],data[ievt][im]['HCAL1_E'][0]]])
                depths = np.concatenate([depths,np.array([2*np.ones(len(data[ievt][im]['HCAL1_x'][0]))]).flatten()])
            
            if (len(data[ievt][im]['HCAL2_x'])>0):
                event = np.concatenate([event,np.c_[data[ievt][im]['HCAL2_x'][0],data[ievt][im]['HCAL2_y'][0],data[ievt][im]['HCAL2_E'][0]]])
                depths = np.concatenate([depths,np.array([3*np.ones(len(data[ievt][im]['HCAL2_x'][0]))]).flatten()])
                
            if (len(data[ievt][im]['HCAL3_x'])>0):
                event = np.concatenate([event,np.c_[data[ievt][im]['HCAL3_x'][0],data[ievt][im]['HCAL3_y'][0],data[ievt][im]['HCAL3_E'][0]]])
                depths = np.concatenate([depths,np.array([4*np.ones(len(data[ievt][im]['HCAL3_x'][0]))]).flatten()])
            
            if (ievt == im == 0):
                print("HCAL X:", np.shape(data[ievt][im]['HCAL3_x']), "+ HCAL Y", np.shape(data[ievt][im]['HCAL3_y']), 
                      "+ HCAL E",np.shape(data[ievt][im]['HCAL3_E']),"= ", "-> np.append to ECAL ->",np.shape(event))
            
            event = np.insert(event, 3, depths,axis=1)
            event = np.insert(event, 4, data[ievt][im]['boundary'][0]*np.ones(len(event)),axis=1)
            #FIXME: use num_global_features Number of additional features to be 
            #concatenated with the latent space observables to form the input to F.
            event = np.insert(event, 5, data[ievt][im]['boundary'][1]*np.ones(len(event)),axis=1)
            
            #X += [event]
            X.append(event)
            #np.append(X,event)
            trueenergy = mc_truth.item().get('true_energy') #TARGET, right?
            Y += [trueenergy]

            
X.end_list()    
#X = np.array(X)
Y = np.array(Y)
print("Y shape =",np.shape(Y),"[MC Truth Energy]")

number of events:  1000
number of "images" per event:  400
ECAL X: (311,) + ECAL Y (311,) + ECAL E (311,) = (311, 3) [using np.c_]
HCAL X: (1, 169) + HCAL Y (1, 169) + HCAL E (1, 169) =  -> np.append to ECAL -> (591, 3)


In [73]:
ncell_max = 1200 #Root file indicase avg. of ~270 per ECal, per HCal, event. 
cell_axis = 2
fill_val = 0

In [83]:
#print(ak.to_list(X.snapshot()))
#print(ak.max(X))
#ak.count(X)

#ak.to_numpy(ak.pad_none(X, ncell_max, axis=cell_axis,clip=True)) #padding awkward array
#ak.fill_none(ak.pad_none(X, ncell_max, axis=cell_axis, clip=True), fill_val)
#testX = ak.fill_none(ak.pad_none(X, ncell_max, axis=cell_axis, clip=True), 999)
#Stack: np.asarray(ak.fill_none(ak.pad_none(X, 2, clip=True), 999))

padded = ak.pad_none(ak.ArrayBuilder.snapshot(X), ncell_max, axis=cell_axis, clip=True)
none_to_0 = ak.fill_none(padded,fill_val,axis=cell_axis)
#X = np.asarray(ak.fill_none(ak.pad_none(ak.ArrayBuilder.snapshot(X), ncell_max, axis=cell_axis, clip=True), fill_val,axis=cell_axis))
print(none_to_0[0])
arr = ak.to_numpy(none_to_0)
#X = np.squeeze(X)
#print("X shape =",np.shape(X),"([Images X Events][Cells][XYEDBB])") #X,Y,Energy,Depth,Boundary,Boundary

[[[105, 955, 22, 1, 100, 100], [30, -1.82e+03, ... 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


ValueError: cannot convert <UnionArray8_64>
    <tags><Index8 i="[0 0 0 0 0 0 0 0 0 0 ... 1 1 1 1 1 1 1 1 1 1]" offset="0" length="960000" at="0x55d4fcb11fe0"/></tags>
    <index><Index64 i="[0 1 2 3 4 5 6 7 8 9 ... 0 0 0 0 0 0 0 0 0 0]" offset="0" length="960000" at="0x55d59d42a400"/></index>
    <content tag="0">
        <ListOffsetArray64>
            <offsets><Index64 i="[0 6 12 18 24 30 36 42 48 54 ... 2332746 2332752 2332758 2332764 2332770 2332776 2332782 2332788 2332794 2332800]" offset="0" length="388801" at="0x55d59ff15c30"/></offsets>
            <content><NumpyArray format="d" shape="2332800" data="105 955 21.9835 1 100 ... -631.319 35.7056 4 480 480" at="0x55d59c25dff0"/></content>
        </ListOffsetArray64>
    </content>
    <content tag="1">
        <NumpyArray format="l" shape="1" data="0" at="0x55d497c7ad20"/>
    </content>
</UnionArray8_64> into np.ndarray

(https://github.com/scikit-hep/awkward-1.0/blob/1.7.0/src/awkward/operations/convert.py#L276)

In [79]:
print(X[0][0][590][2]) #take a look at padding...
print(none_to_0[0][3][590][2])
none_to_0
padded_none
#ak.to_list(X.snapshot())
#X
#ak.ArrayBuilder.snapshot(X)

1.125899906842624e+18
1.125899906842624e+18


<Array [... None, None, None, None, None]]] type='1 * var * 1200 * option[var * ...'>

In [62]:
<Array [... -631, 35.7, 4, 480, 480]]]] type='1 * var * var * var * float64'>
<ArrayBuilder [... -631, 35.7, 4, 480, 480]]]] type='1 * var * var * var * float64'>

ValueError: cannot convert <UnionArray8_64>
    <tags><Index8 i="[0 0 0 0 0 0 0 0 0 0 ... 1 1 1 1 1 1 1 1 1 1]" offset="0" length="960000" at="0x55d4f5e83080"/></tags>
    <index><Index64 i="[0 1 2 3 4 5 6 7 8 9 ... 0 0 0 0 0 0 0 0 0 0]" offset="0" length="960000" at="0x55d4f5f6d690"/></index>
    <content tag="0">
        <ListOffsetArray64>
            <offsets><Index64 i="[0 6 12 18 24 30 36 42 48 54 ... 2332746 2332752 2332758 2332764 2332770 2332776 2332782 2332788 2332794 2332800]" offset="0" length="388801" at="0x55d5843d60c0"/></offsets>
            <content><NumpyArray format="d" shape="2332800" data="105 955 21.9835 1 100 ... -631.319 35.7056 4 480 480" at="0x55d4f6fdb160"/></content>
        </ListOffsetArray64>
    </content>
    <content tag="1">
        <NumpyArray format="l" shape="1" data="0" at="0x55d4c43e2e20"/>
    </content>
</UnionArray8_64> into np.ndarray

(https://github.com/scikit-hep/awkward-1.0/blob/1.7.0/src/awkward/operations/convert.py#L276)

In [ ]:
import energyflow as ef
from energyflow.archs import PFN
from energyflow.utils import data_split
#see https://energyflow.network/examples/

In [ ]:
import tensorflow as tf
tf.keras.utils.normalize(X, axis=-1, order=1)

In [ ]:
X =  np.copy(X) # workaround: output array is read-only
for x in X:
    xy_avg = np.average(x[:,0:2], axis=0)
    x[:,0:2] -= xy_avg
    x[:,2] /= 100. #could make this smarter
    x[:,4:6] /= 100.
    
#QUESTION: What's the ultimate goal of this normalization? Can we use tensorflow built in normalization tools?


In [ ]:
print(X[3000][1000][2])

In [ ]:
(X_train, X_val, X_test,
 Y_train, Y_val, Y_test) = data_split(X, Y, val=0.2, test=0.3)

In [ ]:
# Probably want to standardize the input and output energies, or at least put them in units where the mean is O(1)

In [ ]:
Phi_sizes, F_sizes = (100, 100, 128), (100, 100, 100)
output_act, output_dim = 'linear', 1
loss = 'mse' #mean-squared error
pfn = PFN(input_dim=X.shape[-1], Phi_sizes=Phi_sizes, F_sizes=F_sizes, 
          output_act=output_act, output_dim=output_dim, loss=loss)

In [ ]:
pfn.fit(X_train, Y_train,
        epochs=100,
        batch_size=100,
        validation_data=(X_val, Y_val),
        verbose=1)

In [ ]:
pfn.layers
mypreds = pfn.predict(X_test,batch_size=100)
print(np.shape(mypreds))

In [ ]:
plt.scatter(Y_test,mypreds)

In [ ]:
#????????!

In [ ]:
print(Y_test)
print(mypreds[:,0])